<a href="https://colab.research.google.com/github/anaEver/DL/blob/main/Author_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Data Preparation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import io
import zipfile
from sklearn import preprocessing
!pip install emoji --upgrade
#nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from google.colab import files
from nltk.corpus import stopwords

#stop_words = stopwords.words('english')

import tensorflow as tf
print(tf.__version__)

2.4.0


In [2]:
url = 'https://github.com/Hsankesara/The-Tweets-of-Wisdom/raw/master/tweets.csv'
#url = 'https://github.com/anaEver/DL/blob/main/export_dataframe.csv'
df = pd.read_csv(url)


In [5]:
df.drop(columns=['created_at','likes', 'retweets','handle'],inplace=True)
df.head()

,author_name,handle,tweet_content
0,Naval,naval,"Unresolved thoughts, prematurely pushed out of..."
1,Naval,naval,The modern mind is overstimulated and the mode...
2,Naval,naval,The Lindy Effect for startups:\n\nThe longer y...
3,Naval,naval,@orangebook_ This was a good tweet.
4,Naval,naval,Social media lowers the cost of raising &amp; ...


In [10]:
df.dropna('index',inplace=True)

In [14]:
df = df.groupby('author_name').filter(lambda x : len(x)>50)

In [17]:
#preprocess - tokenize,stemming, punctuation and smiley removal, capitalization, slang and abbreviations
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
import re
from pandas import DataFrame
import emoji

tweet = df.tweet_content.astype(str)
i = 0
#remove url and user mentions
for t in tweet:
  t = emoji.demojize(t)
  t = re.sub('@[A-Za-z0-9_]+','', t)
  t = re.sub('<q>','', t)
  t = re.sub('</q>', '', t)
  t = re.sub("can't",'cannot',t)
  t = re.sub("n't",' not', t)
  t = re.sub("I'm",'I am',t)
  t = re.sub(',',' ',t)
  t =  re.sub(r'http\S+', '', t)# to remove links that start with HTTP/HTTPS in the tweet
  #t  = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', '', t) # to remove other url links
  t = re.sub('[^0-9a-z #+_]',' ',t)#remove numbers
  t = re.sub(r"[^a-zA-Z0-9]+", ' ', t)#special characters
  t = re.sub('[0-9]+','',t)
  df.loc[i, 'tweet_content'] = t
  i += 1
  #print(t)




In [21]:
df.author_name = df.author_name.astype(str)

In [24]:
df.to_csv (r'./tweets.csv', index = False, header=True)
%ls

sample_data/  tweets.csv


In [22]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(df.author_name.values)
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40])

In [23]:
df.tweet_content.nunique()

27743

# GloVe

In [ ]:
!wget http://nlp.stanford.edu/data/glove.twitter.27B.zip
!unzip glove.twitter.27B.zip

In [28]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    verbose = True
    i = 0
    for sentence in tqdm(sentences, disable = (not verbose)):
      for word in sentence:
        try:
          vocab[word] += 1
        except KeyError:
          vocab[word] = 1
          i += 1
    return vocab

In [29]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
sentences = df["tweet_content"].progress_apply(lambda x: x.split()).values
vocab = build_vocab(sentences)
print(len(vocab))

100%|██████████| 24617/24617 [00:00<00:00, 145558.89it/s]

23972


In [27]:
print('Indexing word vectors. 100d')

embeddings_index_100 = {}
f = open('glove.twitter.27B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index_100[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index_100))

Indexing word vectors. 100d
Found 1193514 word vectors.


In [30]:
VOCABULARY_SIZE = 20000
MAX_LENGTH = 140
df.tweet_content  = df.tweet_content.astype(str)
xtrain, xtest, ytrain, ytest = train_test_split(df.tweet_content.values, labels, 
  
 random_state=42, 
 test_size=0.2, shuffle=True)
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(list(xtrain) + list(xtest))

In [31]:
xtrain_sequence = tokenizer.texts_to_sequences(xtrain)
xtest_sequence = tokenizer.texts_to_sequences(xtest)

In [32]:
xtrain_padding = sequence.pad_sequences(xtrain_sequence, maxlen=MAX_LENGTH)
xtest_padding = sequence.pad_sequences(xtest_sequence, maxlen=MAX_LENGTH)
word_index = tokenizer.word_index

In [33]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in tqdm(word_index.items()):
  embedding_vector = embeddings_index_100.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

100%|██████████| 23972/23972 [00:00<00:00, 311483.24it/s]


# LSTM

In [39]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
 100,
 weights=[embedding_matrix],
 input_length=MAX_LENGTH,
 trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(40))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
ytrain_encode = np_utils.to_categorical(ytrain, 40)
ytest_encode = np_utils.to_categorical(ytest, 40)

In [42]:
cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1,
 100,
 weights=[embedding_matrix],
 input_length=MAX_LENGTH,
 trainable=False))
cnn_model.add(Conv1D(filters=100, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
cnn_model.add(Dense(1024, activation='relu'))
cnn_model.add(Dropout(0.8))
cnn_model.add(Dense(1024, activation='relu'))
cnn_model.add(Dropout(0.8))
cnn_model.add(Dense(40))
cnn_model.add(Activation('softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:

history = model.fit(xtrain_padding, y=ytrain_encode, batch_size=512, epochs=20, verbose=1, validation_data=(xtest_padding, ytest_encode))

Epoch 1/20
39/39 [==============================] - 165s 4s/step - loss: 3.5458 - accuracy: 0.0604 - val_loss: 3.3657 - val_accuracy: 0.1026
Epoch 2/20
39/39 [==============================] - 156s 4s/step - loss: 3.3824 - accuracy: 0.0871 - val_loss: 3.2705 - val_accuracy: 0.1336
Epoch 3/20
39/39 [==============================] - 153s 4s/step - loss: 3.2825 - accuracy: 0.1211 - val_loss: 3.1167 - val_accuracy: 0.1629
Epoch 4/20
39/39 [==============================] - 155s 4s/step - loss: 3.1756 - accuracy: 0.1474 - val_loss: 3.0226 - val_accuracy: 0.1885
Epoch 5/20
39/39 [==============================] - 155s 4s/step - loss: 3.1176 - accuracy: 0.1727 - val_loss: 2.9569 - val_accuracy: 0.2061
Epoch 6/20
39/39 [==============================] - 154s 4s/step - loss: 3.0556 - accuracy: 0.1782 - val_loss: 2.9081 - val_accuracy: 0.2132
Epoch 7/20
39/39 [==============================] - 154s 4s/step - loss: 2.9953 - accuracy: 0.1973 - val_loss: 2.8623 - val_accuracy: 0.2291
Epoch 8/20
39

In [43]:

history = cnn_model.fit(xtrain_padding, y=ytrain_encode, batch_size=512, epochs=20, verbose=1, validation_data=(xtest_padding, ytest_encode))

Epoch 1/20
39/39 [==============================] - 106s 3s/step - loss: 3.5353 - accuracy: 0.0672 - val_loss: 3.3598 - val_accuracy: 0.1054
Epoch 2/20
39/39 [==============================] - 100s 3s/step - loss: 3.3440 - accuracy: 0.1021 - val_loss: 3.1160 - val_accuracy: 0.1635
Epoch 3/20
39/39 [==============================] - 100s 3s/step - loss: 3.1377 - accuracy: 0.1583 - val_loss: 2.9577 - val_accuracy: 0.1946
Epoch 4/20
39/39 [==============================] - 100s 3s/step - loss: 3.0045 - accuracy: 0.1865 - val_loss: 2.8594 - val_accuracy: 0.2268
Epoch 5/20
39/39 [==============================] - 100s 3s/step - loss: 2.8812 - accuracy: 0.2163 - val_loss: 2.7716 - val_accuracy: 0.2520
Epoch 6/20
39/39 [==============================] - 101s 3s/step - loss: 2.8040 - accuracy: 0.2365 - val_loss: 2.7167 - val_accuracy: 0.2553
Epoch 7/20
39/39 [==============================] - 100s 3s/step - loss: 2.7454 - accuracy: 0.2549 - val_loss: 2.6599 - val_accuracy: 0.2685
Epoch 8/20
39